In [1]:
class Node:
    
    node_list = list()
    file = None
    
    
    def __init__(self, name=None, ip=None, ty=None):
        self.name = name
        self.ip = ip
        self.type = ty
    def start_tcp_server(self):
        pass
    def start_udp_server(self):
        pass
    
    def __download_file(self, name:str):
        pass
    def __create_get_message(self):
        pass
    def __create_discovery_message(self):
        pass
    def __create_response_message(self):
        pass
    def __send_discover(self, discovery):
        pass
    def __update_list(self, node_list):
        pass
    def __serialize_data(self, s_date):
        pass
    def __deserialize_data(self, s_data):
        pass
    def __handle_response(self):
        pass
    def __save_file(self):
        pass
    def __chose_best(self):
        pass
    def __reassemble_file(self):
        pass
    

In [18]:
class NetWolf:
    """
    This is main class of NetWolf project
    date: 6/8/2020
    author: Amirhosein Amir Firouzkouhi ( 9528007)
    """
    
    def start(self):
        print('NetWolf 1398-1399')
        port = int(input('Please enter port number: '))
        dir = input('Please enter directory of list: ')
    def stop(self):
        pass
    def __start_user_command(self):
        pass
    def __str__(self):
        return "Net wolf < version 1>"


IndentationError: expected an indented block (<ipython-input-18-7217ed1b38e3>, line 12)

In [2]:
import os
def new_name_file(name:str , path:str ):
    index = 0
    while name in os.listdir(path):
        index += 1
        if index == 1:
            dot = name.rfind('.', 0, len(name))
            name = name[:dot] + f'({index})' + name[dot:]
        else:
            name = name.replace(f'({index-1})', f'({index})')
        
    
    return name

def separate_to_mb(data_array:bytearray):
    mb = list()
    mb_size = 10**6
    while True:
        if data_array.__len__()>= mb_size:
            mb.append(data_array[0:mb_size])
            data_array = data[mb_size:]
        else:
            mb.append(data_array)
            break
    return mb

class File:
    
    def __init__(self, name=None, path=None):
        self.name = name
        self.path = path
        
    def save_data(self, data:bytearray):
        file = self._open_file(self.name, self.path)
        
        file.write(data)
        
        self._close_file(file)
        
    def save_list_of_data(self, data_list):
        file = self._open_file(self.name, self.path)
        
        for data in data_list:
            file.write(data)
        
        self._close_file(file)
            
    def _close_file(self, file):
        file.close()
        
    def _open_file(self, name:str, path:str):

        name = new_name_file(name, path)
        __f = open(path + '\\' + name, 'xb')
                    
        return __f
               
    def __str__(self):
        pass

In [5]:
class Report:
    
    def __init__(self,name=None,path=None):
        self.name = name
        self.path = path
        
    def open_report(self):
        pass
    def close_report(self):
        pass
    def write_line(self):
        pass
    def erase_line(self):
        pass

In [36]:

import threading
import socket
import os
from concurrent.futures import ThreadPoolExecutor
from math import ceil

class Server(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        
    def run(self):
        self.__start_server()
    def __start_server(self):
        raise NotImplementedError("Subclass must implement this abstract method")
    def __client_handler(self, socket_info):
        raise NotImplementedError("Subclass must implement this abstract method")
        

def get_ith_mb_from(addr, number):
    file = open(addr, 'rb')

    file.seek((10**6)*number,0)
    temp = file.read(10**6)
    file.close()
    # print(temp)
    return temp

class TcpServer(Server):
    
    
    __ENCODE_MODE = 'utf-8'
    __ERROR_ENCODING = 'backslashreplace'
    __pool = ThreadPoolExecutor(max_workers = 5)
    __tcp_socket = None
    
    # its better to give it address of localhost
    def __init__(self, path:str, addr = None, port = 0):
        Server.__init__(self)
        self.path = path
        self.host_info = (addr, port)
        
    def run(self):
        self.__start_server()
    
    def __start_server(self):
        # get local address
        addr = self.host_info[0]
        port = self.host_info[1]
        if addr is None :
            addr = socket.gethostbyname(socket.gethostname())
        
        # create a socket
        self.__tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # bind a socket to the address and random port
        self.__tcp_socket.bind((addr, port))
        
        # save socket information
        self.host_info = self.__tcp_socket.getsockname()
        
        # start to listening
        self.__tcp_socket.listen(2)
        
        while True:
            temp_socket = self.__tcp_socket.accept()
            self.__pool.submit(self.__client_handler, temp_socket[0])
        
            
    def __client_handler(self, skt:socket.socket):
        siz = int.from_bytes(skt.recv(1), 'big', signed=False)
        cmd_name = skt.recv(siz)
        cmd_name = cmd_name.decode(self.__ENCODE_MODE, self.__ERROR_ENCODING)
        
        if cmd_name == 'GET':
            name = self.__find_file(skt)
            
            if not name is None :
                self.__send(skt, name)
            else:
                self.__response_not_found(skt,name)
        
    def __send(self, skt:socket.socket, name:str):
        file = open(name, 'rb')
        number = ceil(file.seek(0,2)/(10**6))
        file.close()
        
        for part in range(number):
            # use get_ith_mb_from function
            data_file = get_ith_mb_from(name, number)
            
            resp = ResponseData(data_file)
            skt.send(resp.get_data())
            
        # send final response
        resp_done = bytearray(f'done {name}', self.__ENCODE_MODE, self.__ERROR_ENCODING)
        skt.send(ResponseData(resp_done).get_data())
    
    def __find_file(self, skt:socket.socket):
        siz = int.from_bytes(skt.recv(1), 'big', signed=False)
        name = skt.recv(siz).decode(self.__ENCODE_MODE, self.__ERROR_ENCODING)
        
        if name in os.listdir(self.path):
            return self.path + '\\' + name
        else:
            return None
        
    def __response_not_found(self, skt:socket.socket, name):
        data_res = bytearray(f'not found {name}',self.__ENCODE_MODE, self.__ERROR_ENCODING)
        resp_nf = ResponseData(data_res)
        skt.send(resp_nf)


In [ ]:
import threading
import socket
import os
from concurrent.futures import ThreadPoolExecutor
from math import ceil

class UdpServer(Server):
    
    def __init__(self, path:str, addr = None, port = 0):
        Server.__init__(self)
        self.path = path
        self.host_info = (addr, port)
    
    def run(self):
        self.__start_server()
        
    def start(self):
        addr = self.host_info[0]
        port = self.host_info[1]
        if addr is None :
            addr = socket.gethostbyname(socket.gethostname())
        
        self.__udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.host_info = self.__udp_socket.getsockname()
        
        self.__udp_socket.bind((addr, port))
        
        while True:
            siz, dest = self.__udp_socket.recvfrom(1)
        
        
    def __client_handler(self, socket = None):
        pass    

In [4]:
def test_File_class():
    addr = 'League Of Legends - Awaken Ft. Valerie Broussard (Official Audio) - .mp3'
    f = File('lg.mp3', 'F:\\Amir\\University\\Computer Network\\Project\\NetWolf')
    i = open(addr, 'rb')
    temp = bytearray(i.read(-1))
    print(temp.__len__())
    mb = separate_to_mb(temp)
    
    f.save_list_of_data(mb)
    
    i.close()

test_File_class()

7902383


TypeError: listdir: path should be string, bytes, os.PathLike or None, not module

In [47]:
b = bytearray('hello', 'utf-8')
a = bytearray('get', 'utf-8')
c = a+b
print(c)
print(b'\x1b\x51'.__len__())

bytearray(b'gethello')
2


In [90]:

class Message:
    
    command = '<command>'
    def __init__(self,data:bytearray):
        self.data = data
        
    def get_data(self):
        raise NotImplementedError("Subclass must implement this abstract method")   
    def __str__(self):
        temp = bytearray(self.command, 'utf-8', 'backslashreplace')
        return 'size: {sc}\n' \
               'command: {c}\n' \
               'size of data: {sd}\n' \
               'data: {d}'.format(sc=len(temp), c=self.command, sd=len(self.data), d=self.data)
    

class GetData(Message):
    
    def __init__(self,data):
        Message.__init__(self, data)
        self.command = 'GET'
        
    def get_data(self):
        """
        size - command - size of data - data
        1B   -   sizeB -      2B      - size of data B
        """
        temp = bytearray(self.command,'utf-8', 'backslashreplace')
        packet = bytearray([len(temp)]) + temp
        packet += len(self.data)
        packet += self.data
        
        return packet


class ResponseData(Message):
    
    def __init__(self,data):
        Message.__init__(self, data)
        self.command = 'RSP'
        
    def get_data(self):
        """
        size - command - size of data - data
        1B   -   sizeB -      2B      - size of data B
        """
        temp = bytearray(self.command,'utf-8', 'backslashreplace')
        packet = bytearray([len(temp)]) + temp
        packet += len(self.data)
        packet += self.data
        
        return packet

class DirectoryData(Message):
    
    def __init__(self,data):
        Message.__init__(self, data)
        self.command = 'DIR'
        
    def get_data(self):
        """
        size - command - size of data - data
        1B   -   sizeB -      2B      - size of data B
        """
        temp = bytearray(self.command,'utf-8', 'backslashreplace')
        packet = bytearray([len(temp)]) + temp
        packet += len(self.data)
        packet += self.data
        
        return packet  
                
        

In [50]:
msgFromServer       = "Hello UDP Client"

bytesToSend         = str.encode(msgFromServer)
print(bytesToSend.__len__())

16


In [56]:
int.to_bytes(255,1,'big')

b'\xff'

In [59]:
import socket
help(socket.socket.recvfrom)

Help on method_descriptor:

recvfrom(...)
    recvfrom(buffersize[, flags]) -> (data, address info)
    
    Like recv(buffersize, flags) but also return the sender's address info.



In [64]:
command = 'GET'
command_size = command.encode('utf-8', 'backslashreplace') .__len__()
print(command_size)

3


3

In [97]:
data = bytearray('hello','utf-8', 'backslashreplace')
a = GetData(data)
b = DirectoryData(data)
c = ResponseData(data)
print(a)
print(b)
print(c)




size: 3
command: GET
size of data: 5
data: bytearray(b'hello')

size: 3
command: DIR
size of data: 5
data: bytearray(b'hello')

size: 3
command: RSP
size of data: 5
data: bytearray(b'hello')



In [96]:
int.to_bytes(1024, 2, 'big', signed=False)
int.from_bytes(b'\x04\x00','big', signed=False)

1024

In [101]:
help(socket.socket.getsockname)





Help on method_descriptor:

getsockname(...)
    getsockname() -> address info
    
    Return the address of the local endpoint.  For IP sockets, the address
    info is a pair (hostaddr, port).



In [108]:
from random import shuffle
f = open('F:\\textfile.txt', 'wt')
for i in range(1000):
    f.write(f'012345678'*10 + '\n')
    
f.close()

In [113]:
from random import shuffle
l = list(range(0,10))
shuffle(l)
print(l)
str(list)




[8, 7, 4, 6, 3, 5, 2, 0, 9, 1]


"<class 'list'>"

In [5]:
import socket
print(socket.gethostname())
print(socket.gethostbyname(socket.gethostname()))
help()

DESKTOP-C1DQMIK
192.168.56.1

Welcome to Python 3.7's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the Internet at https://docs.python.org/3.7/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".


You are now leaving help and returning to the Python interpreter.
If you want to ask for help on a particular object directly from the
interpreter, you can type "help(object)".  Executing "help('string')"
has the same effect as typing a particular string at the help> prompt.


In [37]:
import os
from math import ceil
# print(os.listdir())
addrr='lg.mp3'
m = open(addrr, 'rb')
n = ceil(m.seek(0,2)/(10**6))
f = File('part.mp3','F:\\Amir\\University\\Computer Network\\Project\\NetWolf')
for i in range(n):
    b = get_ith_mb_from(addrr, i)
    f.save_data(b)

m.close()




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
import os
print(os.getcwd())

F:\Amir\University\Computer Network\Project\NetWolf


In [ ]:
l = os.listdir('F:\\Amir\\University\\Computer Network\\Project\\NetWolf')
f = open('music1.mp3','wb')
for i in range(8):
    if f'part({i}).mp3' in l:
        t = open(f'part({i}).mp3', 'rb')
        f.write(t.read())
        t.close()
        print(i)

f.close()


Help on built-in function read:

read(...) method of _io.BufferedWriter instance
    Read and return up to n bytes.
    
    If the argument is omitted, None, or negative, reads and
    returns all data until EOF.
    
    If the argument is positive, and the underlying raw stream is
    not 'interactive', multiple raw reads may be issued to satisfy
    the byte count (unless EOF is reached first).  But for
    interactive raw streams (as well as sockets and pipes), at most
    one raw read will be issued, and a short result does not imply
    that EOF is imminent.
    
    Returns an empty bytes object on EOF.
    
    Returns None if the underlying raw stream was open in non-blocking
    mode and no data is available at the moment.

